In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def corr2d(X, K):
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

In [3]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [17]:
# input image
X = torch.ones((6, 8))
X[:, 2:6] = 0
X
# this has vertical edges

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [18]:
# kernel that detects edges where 1 detects white to black and -1 detects black to white
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

In [19]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [13]:
# now we find the transpose's edges which are vertical
Y = corr2d(X.t(), K)
Y

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

As expected we see all 0s since there are no vertical edges here

In [20]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = nn.LazyConv2d(1, kernel_size=(1, 2), bias=False)

# The two-dimensional convolutional layer uses four-dimensional input and
# output in the format of (example, channel, height, width), where the batch
# size (number of examples in the batch) and the number of channels are both 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # Learning rate

for i in range(10):
    Y_hat = conv2d(X)
    # print(Y_hat)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # Update the kernel
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')

tensor([[[[0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317],
          [0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317],
          [0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317],
          [0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317],
          [0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317],
          [0.3317, 0.3211, 0.0000, 0.0000, 0.0000, 0.0105, 0.3317]]]],
       grad_fn=<ConvolutionBackward0>)
tensor([[[[-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653],
          [-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653],
          [-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653],
          [-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653],
          [-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653],
          [-0.2653,  0.3267,  0.0000,  0.0000,  0.0000, -0.5921, -0.2653]]]],
       grad_fn=<ConvolutionBackward0>)
epoch 2, loss 4.563
tensor([[[[ 0.2123,  0.7601,  0.0000, 

In [9]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0113, -0.9623]])